In [1]:
import scipy.io
import numpy as np

In [ ]:
mat_data = scipy.io.loadmat('./data_preprocessed_matlab/s01.mat') 
original_data = mat_data['data']
original_label = mat_data['labels']

In [ ]:
sliced_data = original_data[:, :32, 384:] 
print("sliced_data：", sliced_data.shape) # (40, 32, 7680)

eeg_data = sliced_data

In [ ]:
valence = original_label[:,0]
arousal = original_label[:,1]
# HAHV--1, LAHV--2, LALV--3, HALV--4
VA_labels = np.where((valence > 5) & (arousal > 5), 0,
          np.where((valence >= 5) & (arousal < 5), 1,
            np.where((valence < 5) & (arousal < 5), 2, 3)))
print("V:", valence)
print("A:", arousal)
print(VA_labels)

segment_size = 3 * 128

num_segments = sliced_data.shape[2] // segment_size # 7680/3/128 = 20
expanded_VA_labels = np.repeat(VA_labels, num_segments)
print(expanded_VA_labels.shape) # (800,)
labels = expanded_VA_labels 

In [ ]:
import pandas as pd
import mne
from mne import io
from mne.datasets import sample
from mne.preprocessing import ICA
import os

sfreq = 128 
channels = 32
samples = 384
num_each_trial = 20
ch_names = ['Fp1', 'AF3', 'F7', 'F3', 'FC1', 'FC5', 'T7', 'C3', 'CP1', 'CP5', 'P7', 'P3', 
            'Pz', 'PO3', 'O1', 'Oz', 'O2', 'PO4', 'P4', 'P8', 'CP6', 'CP2', 'C4', 'T8', 
            'FC6', 'FC2', 'F4', 'F8', 'AF4', 'FP2', 'Fz', 'Cz']
ch_types = ['eeg'] * channels

data_list = []
eeg_data_segments = np.split(eeg_data, 40, axis=0) # (40, 32, 7680)
index = 0
for segment in eeg_data_segments:
    index = index + 1
    segment_2d = segment.reshape(-1, channels).T

    info = mne.create_info(ch_names=ch_names, ch_types=ch_types, sfreq=sfreq)
    raw = mne.io.RawArray(segment_2d, info=info)

    raw.filter(l_freq=1.0, h_freq=50.0)

    ica = ICA(n_components=channels, random_state=0, max_iter=1000) 
    ica.fit(raw)

    ica.exclude = [] 
    ica.apply(raw)

    data = raw.get_data().T # (7680, 32)

    data = data[:num_each_trial * samples, :]

    data = data.reshape(num_each_trial, samples, channels)

    data_list.append(data)


data_array = np.concatenate(data_list, axis=0) # (800, 384, 32)
data_array = np.swapaxes(data_array, 1, 2) # (800, 32, 384)
eeg_data = data_array

In [6]:
np.save('./EEGData/s01_eeg.npy', eeg_data) # (800, 32, 384)
np.save('./EEGData/s01_labels.npy', labels) # (800,)